In [1]:
from datetime import date, datetime, timezone

import cftime
import git
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import esmlab
import esmlab_regrid
import pop_tools

import config
import util

In [2]:
# script identifier to go in generated file

repo = 'github.com/marbl-ecosys/marbl-forcing'
sha = git.Repo(search_parent_directories=True).head.object.hexsha
script_fname = 'initial_conditions/gen_cesm2_omip_IC.ipynb'
id_string = f'{repo}/tree/{sha}/{script_fname}'

In [3]:
# input specifications

tracernames = [
    'NO3', 'PO4', 'SiO3', 'O2', 'DIC', 'ALK', 
    'spC', 'spChl', 'spFe', 'spP', 'spCaCO3', 
    'diatC', 'diatChl', 'diatFe', 'diatP', 'diatSi', 
    'diazC', 'diazChl', 'diazFe', 'diazP', 
    'zooC', 
    'DOC', 'DOCr', 'DON', 'DONr', 'DOP', 'DOPr', 'NH4', 
    'Fe', 'Lig'
]

# IC specified by OMIP (macronutrients, O2, DIC, ALK)
dirin_omip = '/glade/p/cesm/bgcwg/OMIPinit'
omip_metadata = {
    'NO3':{'fname':'woa13_nitrate_OMIPinit.nc', 'varname':'no3'},
    'PO4':{'fname':'woa13_phosphate_OMIPinit.nc', 'varname':'po4'},
    'SiO3':{'fname':'woa13_silicate_OMIPinit.nc', 'varname':'si'},
    'O2':{'fname':'woa13_oxygen_OMIPinit.nc', 'varname':'o2'},
    'DIC':{'fname':'glodapv2_Ct_preind_OMIPinit.nc', 'varname':'Ct_preind'},
    'ALK':{'fname':'glodapv2_At_OMIPinit.nc', 'varname':'At'}
}

# Add grid name to metadata.
# This is fragile. It would be better to determine this automatically.
# Alternatively, the script would be more self contained if the grid were constructed on the fly.
omip_metadata['NO3']['grid'] = 'latlon_1x1_180W'
omip_metadata['PO4']['grid'] = 'latlon_1x1_180W'
omip_metadata['SiO3']['grid'] = 'latlon_1x1_180W'
omip_metadata['O2']['grid'] = 'latlon_1x1_180W'
omip_metadata['DIC']['grid'] = 'latlon_1x1_20E'
omip_metadata['ALK']['grid'] = 'latlon_1x1_20E'

# get other tracers (living pools, DOM, NH4, Fe, Lig) from refcase
refcase = 'b.e21.B1850.f09_g17.CMIP6-piControl.001'
refdate = '0501-01-01'
dirin_refcase = f'{config.inputdata}/cesm2_init/{refcase}/{refdate}'
fname_refcase = f'{dirin_refcase}/{refcase}.pop.r.{refdate}-00000.nc'
print(fname_refcase)

/glade/p/cesmdata/cseg/inputdata/cesm2_init/b.e21.B1850.f09_g17.CMIP6-piControl.001/0501-01-01/b.e21.B1850.f09_g17.CMIP6-piControl.001.pop.r.0501-01-01-00000.nc


In [4]:
# details on file being generated

dst_grid = 'POP_gx1v7'
datestamp = date.today().strftime("%y%m%d")
fname_out = f'{config.dirout}/ecosys_jan_IC_omip_{dst_grid}_c{datestamp}.nc'

In [5]:
# start ds_out as Dataset of grid variables

ds_out = pop_tools.get_grid(grid_name=dst_grid)

In [6]:
# generate output grid 3D mask

nk = len(ds_out.z_t)
nj, ni = ds_out.KMT.shape

# make 3D array of 0:km
zero_to_km = xr.DataArray(np.arange(0, nk), dims=('z_t'))
ONES_3d = xr.DataArray(np.ones((nk, nj, ni)), dims=('z_t', 'nlat', 'nlon'))
ZERO_TO_KM = (zero_to_km * ONES_3d)

# mask out cells where k is below KMT
MASK = ZERO_TO_KM.where(ZERO_TO_KM < ds_out.KMT)
MASK = xr.where(MASK.notnull(), True, False)

MASK['z_t'] = ds_out.z_t

In [7]:
# copy from refcase restart file to ds_out tracers not yet in ds_out

fillvalue = -1.0e34 # value to replace land values with

ds_refcase = xr.open_dataset(fname_refcase).rename({'k':'z_t', 'j':'nlat', 'i':'nlon'})
for tracername in tracernames:
    if tracername not in ds_refcase.data_vars:
        print(f'copying {tracername} from {refcase} restart file')
        tracer = ds_refcase[tracername+'_CUR']
        tracer = tracer.where(MASK)
        # Set floor values that are 0.0 to value 1 layer shallower.
        # This is to fill in values where KMT was popped up by overflow parameterization.
        # This is needed to enable using ds_out in runs where overflows are disabled.
        for k in range(1, ds_out.dims['z_t']):
            tracer[k, :, :] = xr.where((tracer[k, :, :] == 0.0) & (k+1 == ds_out.KMT),
                                       tracer[k-1, :, :], tracer[k, :, :])
        tracer.attrs['coordinates'] = 'TLONG TLAT z_t'
        tracer.encoding['_FillValue'] = fillvalue
        ds_out[tracername] = tracer

copying NO3 from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying PO4 from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying SiO3 from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying O2 from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying DIC from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying ALK from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spC from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spChl from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spFe from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spP from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spCaCO3 from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatC from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatChl from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatFe from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copy

In [8]:
# set derived quantities

ds_out['DIC_ALT_CO2'] = ds_out['DIC']
ds_out['ALK_ALT_CO2'] = ds_out['ALK']
DOCtot = ds_out['DOC']
DOCtot.values = ds_out['DOC'].values + ds_out['DOCr'].values
DOCtot.attrs['long_name'] = 'DOC + DOCr'
ds_out['DOCtot'] = DOCtot

In [9]:
# write generated file

datestamp = datetime.now(timezone.utc).strftime("%Y-%m-%d")
ds_out.attrs['history'] = f'created by {id_string} on {datestamp}'
omip_file_list = [dirin_omip+'/'+omip_metadata[varname]['fname'] for varname in omip_metadata]
ds_out.attrs['input_file_list'] = ' '.join(omip_file_list + [fname_refcase])
util.ds_clean(ds_out).to_netcdf(fname_out)